In [1]:
import numpy as np
import pandas as pd
import models
import warnings
warnings.simplefilter('ignore')
import math
from sklearn.pipeline import Pipeline
from sklearn import set_config
from matplotlib import style
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

In [2]:
hankel = models.Hankel()
rank = models.Rank()
cluster = models.Cluster()
meepc = models.MEEPC() 

In [3]:
df = pd.read_csv('~/data/training_data.csv')

In [4]:
df

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,...,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LT_001_PV,LEAK_DIFF_PRESSURE,TOTAL_CONS_REQUIRED_FLOW,"Attack LABLE (1:No Attack, -1:Attack)"
0,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
1,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
2,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
3,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
4,167.895,0.619473,11.6179,504.145,0.293009,0.001080,62.1472,1.0,1.0,1.0,...,0.0,8279.1,14.00090,264.802,0.620240,0.672394,67.1004,60.1557,0.11,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971995,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971996,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971997,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971998,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0


In [5]:
df_normal=df[df['Attack LABLE (1:No Attack, -1:Attack)']==1]
df_attack=df[df['Attack LABLE (1:No Attack, -1:Attack)']==-1]

In [6]:
df_normal

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,...,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LT_001_PV,LEAK_DIFF_PRESSURE,TOTAL_CONS_REQUIRED_FLOW,"Attack LABLE (1:No Attack, -1:Attack)"
0,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
1,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
2,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
3,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
4,167.895,0.619473,11.6179,504.145,0.293009,0.001080,62.1472,1.0,1.0,1.0,...,0.0,8279.1,14.00090,264.802,0.620240,0.672394,67.1004,60.1557,0.11,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971938,179.491,0.631472,11.9234,420.706,0.228469,1.899090,42.5462,2.0,1.0,1.0,...,0.0,8279.1,7.05323,690.042,0.516764,0.001354,64.3759,68.7421,1.56,1.0
971939,179.491,0.631472,11.9234,420.706,0.228469,1.899090,42.5462,2.0,1.0,1.0,...,0.0,8279.1,7.05323,690.042,0.516764,0.001354,64.3759,68.7421,1.56,1.0
971940,179.491,0.631472,11.9234,420.706,0.228469,1.899090,42.5462,2.0,1.0,1.0,...,0.0,8279.1,7.05323,690.042,0.516764,0.001354,64.3759,68.7421,1.56,1.0
971941,179.491,0.631472,11.9234,420.706,0.228469,1.899090,42.5462,2.0,1.0,1.0,...,0.0,8279.1,7.05323,690.042,0.516764,0.001354,64.3759,68.7421,1.56,1.0


In [7]:
sensors = [3,9,25,43]
df_normal = df_normal.iloc[:, sensors]
df_attack=df_attack.iloc[:,sensors]

In [8]:
def calc_width_height(v):
    width = 2/np.sqrt(v) 
    return width

    
def draw_ellipse(i,j,centers,widths,X):
    plt.xlim(centers[i]-widths[i]/2,centers[i]+widths[i]/2)
    plt.ylim(centers[j]-widths[j]/2,centers[j]+widths[j]/2)

    ellipse = Ellipse(xy=(centers[i], centers[j]), width=widths[i], height=widths[j])
    ellipse.fill = False
    ellipse.set_ec('red')
    ax.add_patch(ellipse)
    plt.scatter(x = X[::,i], y = X[::,j])
    

In [9]:
X = df_normal.iloc[:,2].values
X = hankel.fit(X,30,0.5)
X=X.T # size--> 1069X1800
kmeans,optimal_k = cluster.fit(X)
print(optimal_k)
radiis = []
weights=[]
centers=[]
clusters_R=[]
clusters_V=[]
# tsne=TSNE(n_components=2,perplexity=30)


for i in range(optimal_k):
    cluster_ = X[np.where(kmeans.labels_ == i)[0]]
    r = rank.fit(cluster_)
    clusters_R.append(r)
    print("r: "+str(r))
    U,Sigma,VT = np.linalg.svd(cluster_)
    V = VT.T
    clusters_V.append(V)
    cluster_ = np.matmul(cluster_,V[:,:r])
    # cluster_=tsne.fit_transform(cluster_)
    weight,center = meepc.fit(cluster_)
    weights.append(weight)
    centers.append(center)
    widths=calc_width_height(weight)
    var1=np.square(cluster_-center)
    var2=np.matmul(weight,var1.T)
    max_radius=math.sqrt(np.amax(var2))
    radiis.append(max_radius)
    #plotting
#     ax = plt.gca()
#     ellipse = Ellipse(xy=(center[0],center[1]), width=widths[0], height=widths[1])
#     ellipse.fill = False 
#     ellipse.set_ec('red')
#     ax.add_patch(ellipse)
#     sns.scatterplot(x = cluster_[:,0], y = cluster_[:,1])
# plt.show()
radiis



4
r: 14
r: 13
r: 12
r: 12


[1.0014734114254888,
 1.0002201982034173,
 1.0131606656322492,
 1.0003705943179166]

In [10]:
if math.isnan(max_radius):
        print('Rank: ',r)
        print('Cluster Shape: ',cluster.shape)
        print('CLuster: ',cluster)
        print('Weight Shape: ', weight.shape)
        print('Weight: ', weight)
        print('Center Shape: ', center.shape)
        print('Center: ', center)
        print('Var1: ', var1)
        print('Var2: ', var2)

# testing 

In [11]:
df_test=pd.read_csv('~/data/testdata.csv')
df_test

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,...,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LT_001_PV,LEAK_DIFF_PRESSURE,TOTAL_CONS_REQUIRED_FLOW,"Attack LABLE (1:No Attack, -1:Attack)"
0,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
1,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
2,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
3,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
4,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248395,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0
248396,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0
248397,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0
248398,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0


In [12]:
y_actual=df_test.iloc[:,-1]
# df_test_normal=df_test[df_test['Attack LABLE (1:No Attack, -1:Attack)']==1]
sensors = [3,9,25,4]
df_test=df_test.iloc[:,sensors]
df_test


,1_AIT_004_PV,1_MV_003_STATUS,2_FIC_401_CO,1_AIT_005_PV
0,463.716,1.0,20.2507,0.267211
1,463.716,1.0,20.2507,0.267211
2,463.716,1.0,20.2507,0.267211
3,463.716,1.0,20.2507,0.267211
4,463.716,1.0,20.2507,0.267211
...,...,...,...,...
248395,430.680,1.0,19.2170,0.228327
248396,430.680,1.0,19.2170,0.228327
248397,430.680,1.0,19.2170,0.228327
248398,430.680,1.0,19.2170,0.228327


In [13]:
X_test=df_test.iloc[:,2].values
Xtest=hankel.fit(X_test,30,0.5)  #size -> 1800x274
Xtest=Xtest.T  #size -> 274x1800
print(Xtest.shape)
distances=[]
for i in range(optimal_k):
    Xtest_cluster=np.matmul(Xtest,clusters_V[i][:,:clusters_R[i]]) # size -> 274 x R
    var1=np.square(Xtest_cluster-centers[i])
    var2=np.matmul(weights[i],var1.T)
    distance=np.sqrt(var2)
    distances.append(distance)


(274, 1800)


In [14]:
distances[0].shape #size (274,)

<function ndarray.reshape>

In [15]:
# distances matrix
stacked_arr=np.vstack(distances)
distances_arr=np.transpose(stacked_arr)
distances_arr #-> 274 X 4

array([[1.23307772, 1.91183702, 1.88980583, 0.82753116],
       [1.45744475, 0.59516904, 1.52712792, 1.30231661],
       [1.44444121, 0.5280319 , 1.38266788, 1.18354432],
       ...,
       [1.01382955, 1.22403106, 0.40179398, 1.33924728],
       [0.42158724, 1.85588754, 1.09004837, 0.89789537],
       [0.50912986, 1.93499066, 1.08380835, 0.88750514]])

In [16]:
radiis

[1.0014734114254888,
 1.0002201982034173,
 1.0131606656322492,
 1.0003705943179166]

In [17]:
check_anamoly=radiis-distances_arr # positive = inside cluster | negative = outside cluster
check_anamoly

array([[-0.2316043 , -0.91161682, -0.87664516,  0.17283943],
       [-0.45597134,  0.40505116, -0.51396725, -0.30194601],
       [-0.4429678 ,  0.4721883 , -0.36950722, -0.18317372],
       ...,
       [-0.01235614, -0.22381086,  0.61136668, -0.33887669],
       [ 0.57988617, -0.85566734, -0.0768877 ,  0.10247522],
       [ 0.49234356, -0.93477046, -0.07064768,  0.11286546]])

In [18]:
#check for if any datapoint is coming inside nay cluster
is_positive=np.any(check_anamoly>0,axis=1)
#indices of those rows
positive_indices=np.where(is_positive)[0]
print(positive_indices)


[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  89  90
  91  92  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110
 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128
 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146
 148 149 150 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166
 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184
 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202
 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220
 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238
 239 240 241 242 243 244 245 246 247 248 249 250 25

In [19]:
len(positive_indices)

269